# Real time Behaviour Recognition Performance

In [ ]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import jaccard_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
pd.set_option('display.min_rows', 100)

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
import src.utils as utils

palette0 = sns.color_palette(['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000']) # Okabe-Ito
# palette1 = sns.color_palette(["#ff4554", "#00bbdf", "#bad600", "#f02d7d", "#f8b62e", "#8b26a6","#808080"]) # https://anoiro.com/themes/switch-joycons
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette3 = sns.color_palette(["#1E88E5", "#D81B60", "#FFC107", "#004D40"])
palette = palette3
display(palette)
sns.set_palette(palette)
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

# Ground Truth

In [ ]:
path_gt = "../data/ground-truth/umineko_2024_ground_truth.csv"
df_gt = pd.read_csv(path_gt)
print(len(df_gt))
display(df_gt.head(5))
display(df_gt.tail(5))

In [ ]:
df_gt_grouped = df_gt.groupby(['test_id', 'camera_count']).size().reset_index(name='count')
display(df_gt_grouped)

# Prediction

In [ ]:
target_dir = f"../data/extracted-gps-data/"
target_path = f"{target_dir}/*/*.csv"
path_list = sorted(glob.glob(target_path))
print(len(path_list))

columns = [
    'behavior_class', 'camera_command', 'camera_recording', 'camera_count'
]

df = pd.DataFrame()
for i, path in enumerate(path_list):
    test_id = os.path.basename(path)[:5]
    session_id = os.path.basename(path)[6:9]
    df_tmp = pd.read_csv(path)

    # Exception handling
    if (test_id == "LBP03" and session_id == "S00") or (test_id == "LBP08" and session_id in ["S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10", "S11", "S12", "S13"]):
        nan_rows = pd.DataFrame(np.nan, index=[0], columns=df.columns)
        df_tmp = pd.concat([nan_rows, df_tmp], ignore_index=True)
        df_tmp.iloc[0, df_tmp.columns.get_loc('test_id')] = test_id
        df_tmp.iloc[0, df_tmp.columns.get_loc('camera_count')] = int(session_id[1:])
        df_tmp.iloc[1, df_tmp.columns.get_loc('test_id')] = test_id
        df_tmp.iloc[1, df_tmp.columns.get_loc('camera_count')] = int(session_id[1:])
    print(f"{test_id}_{session_id}: {len(df_tmp)}")
    # df_tmp = df_tmp[df_tmp['rtc_msec'] == 0]
    # df_tmp = df_tmp[df_tmp['camera_count'] > -1]
    df_tmp = df_tmp[columns]
    df_tmp.insert(0, 'test_id', [test_id]*len(df_tmp))
    df_tmp.insert(len(df_tmp.columns), 'program_sec', np.arange(1, len(df_tmp)+1, 1))
    df = pd.concat([df, df_tmp], axis=0)

df['old_index'] = df.index
df = df.reset_index(drop=True)

print(len(df))
display(df)

df_pred = df

In [ ]:
df_grouped = df.groupby(['test_id', 'camera_count']).size().reset_index(name='count')
display(df_grouped)

In [ ]:
df_count = pd.concat([df_gt_grouped, df_grouped], axis=1)
df_count.columns = ['test_id_gt', 'camera_count_gt', 'count_gt', 'test_id', 'camera_count', 'count']
df_count.insert(len(df_count.columns), 'diff', df_count['count_gt']-df_count['count'])
display(df_count)

## Field Test

In [ ]:
# check length
print(f"Ground Truth: {len(df_gt)}")
print(f"Prediction: {len(df_pred)}")

In [ ]:
df_eval = df_gt.copy()
df_eval.insert(len(df_eval.columns), 'pred', df_pred['behavior_class'])
display(df_eval.head(10))
df_eval = df_eval.dropna(subset=['label'])
display(df_eval.head(10))

In [ ]:
y_pred = df_eval['pred']
y_true = df_eval['label']
utils.calc_and_report_performance_metrics(y_true, y_pred)

## Online Test

In [ ]:
df1 = pd.read_csv("../data/tree-model/df_y_gt_y_pred_umineko_2024_tree_binary_2.csv")
y_true1, y_pred1 = df1['y_gt'], df1['y_pred']
cm1 = confusion_matrix(y_true1, y_pred1)
utils.calc_and_report_performance_metrics(y_true1, y_pred1)

## Fig. S01 | Online Test & Field Test

In [ ]:
GRIDSPEC_KW = {'width_ratios': [1, 1], 'wspace': 0.5, 'hspace': 0.1}
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5), gridspec_kw=GRIDSPEC_KW)
ax1 = utils.plot_confusion_matrix_ax(ax=ax1, cm=cm1)
ax2 = utils.plot_confusion_matrix_ax(ax=ax2, cm=cm2)

ax1.text(-0.45, -0.2, '(a)', fontsize=24, fontweight='bold', va='top', ha='right')
ax2.text(-0.45, -0.2, '(b)', fontsize=24, fontweight='bold', va='top', ha='right')

plt.show()
plt.close()

save_dir = f"../output/figure-for-paper"
fig.savefig(f"{save_dir}/png/fig_s01_confusion_matrix.png", dpi=350, bbox_inches="tight", pad_inches=0.2, transparent=False)
fig.savefig(f"{save_dir}/pdf/fig_s01_confusion_matrix.pdf", dpi=600, bbox_inches="tight", pad_inches=0.2, transparent=False)
# fig.savefig(f"{save_dir}/fig_s01_confusion_matrix.svg", bbox_inches="tight", pad_inches=0.1, transparent=False)